In [ ]:
from io import BytesIO
import hashlib
import random

import util
from test_framework.key import generate_schnorr_nonce, ECKey, ECPubKey, SECP256K1_FIELD_SIZE
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, sign_musig
from test_framework.script import TapLeaf, TapTree, TaprootSignatureHash
from test_framework.address import program_to_witness
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness
from test_framework.util import assert_equal


# 3.2  Vault with Claw-back Mechanism

In this case-study, we consider how we can use a taproot output and pre-signed transactions to create a simple [claw-back](https://lists.linuxfoundation.org/pipermail/bitcoin-dev/2019-August/017229.html) vault design.

![test](images/vault0.jpg)

Vault Output:
* `2/2 multisig`
    * Signed by 2 ephemeral keys.
    * Spendable by pre-signed delayed-spend transaction child.


Delayed Spending Output:
* `3/5 multisig` 
    * Spendable by wallet keys after 30 days.

* `Re-vaulting Tx Musig Public Key` 
    * Signed with ephemeral private key
    * Spendable by re-vaulting transaction after 10 days.

* `4/7 multisig` 
    * Spendable by wallet keys after 1 days.

* `Ragequite Public Key` 
    * Spendable by "ragequit" key.
 
 
Re-vaulting Transaction Output:
* _Same as Vault Output_
* _Also requires pre-signed delayed-spend transaction child._


***Other Requirements***

* No unused public keys should be revealed during spending.
* Only pre-signed transactions are capable of signing with MuSig.


___

## TODO: Build the claw-back vault with Taproot.
* Implement the clawback vault involving multiple pre-signed transactions. 
* Test the spending paths of the delayed transaction output.
   

**Build your taproot output.**

* Construct the Taptree according to the spending paths and their likelihoods.

In [ ]:
# Generate Vault TX output keypairs.

# Generate Delayed Vault TX Output Keypairs.

# Generate Revaulting TX Output Keypairs.

# Generate Output Keypairs of the Revaulting TX child.

# Build tapscripts & taproots

# WIP

# Construct Segwit Address.
tapscript, taptweak, control_map = multisig_taproot.construct()
output_pubkey = musig_ABC.tweak_add(taptweak) 
output_pubkey_b = output_pubkey.get_bytes()
taproot_pubkey_v1 = bytes([output_pubkey_b[0] & 1]) + output_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print("\nSegwit Address:", segwit_address)


**Test the revaulting spending path.**

* Construct the spending transaction and test the tx validity with the `testmempoolaccept`.
* Demonstrate the delays work intended.
* Compute and compare the weight of each spend.

***Start TestNodes***

In [ ]:
test = util.TestWrapper()
test.setup(num_nodes=1)


***Generate Wallet Balance***

In [ ]:
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)


***Send funds from the wallet to the taproot output (Segwit Address).***

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, 0.5)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet places change output at a random txout index.)
outputs = iter(tx.vout)
taproot_output = next(outputs)
taproot_index = 0

while (taproot_output.scriptPubKey != tapscript):
    taproot_output = next(outputs)
    taproot_index += 1
taproot_value = taproot_output.nValue


***Test taproot spend for all script paths***

In [ ]:
# WIP

***Shutdown TestNode***

In [ ]:
test.shutdown()
